<a href="https://colab.research.google.com/github/julissaponte/CodeChallenge/blob/main/Algoritmo_Regresion_Lineal_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [153]:
df = pd.read_csv('/ObesityDataSet_raw_and_data_sinthetic.csv')

train = df[:2000]
test = df[2000:]

train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

VARIABLES POR INDICE DE MASA CORPORAL

IMC = ALTURA / PESO^2

Debajo del peso ideal: Menos de 18.5

Normal: 18.5 - 24.9

Sobrepeso: 25.0 - 29.9

Obesidad 1: 30.0 - 34.9

Obesidad 2: 35.0 - 39.9

Obesidad 3: Más de 40

**Atributos relacionados con el peso:**

- *Calories consumption monitoring (SCC)*
-- YES
-- NO


- *Physical activity frequency (FAF)*
-- NUMERIC

- *Time using technology devices (TUE)*
-- NUMERIC

- *Transportation used (MTRANS)*
-- AUTOMOBILE
-- MOTORBIKE
-- BIKE
-- PUBLIC_TRANSPORTATION
-- WALKING

-11 <= BEGINNER <= -7

-6 <= INTERMEDIATE <= -2

-1 <= EXPERT <= 3

In [180]:
estadoxvalores = {
    "Insufficient_Weight": 0,
    "Normal_Weight": 0,
    "Overweight_Level_I": -1,
    "Overweight_Level_II": -2,
    "Obesity_Type_I": -3,
    "Obesity_Type_II": -4,
    "Obesity_Type_III": -5,
}
FAFxvalores = {
    3 : 1,
    2 : 0,
    1 : -1,
    0 : -2,
}
TUExvalores = {
    0 : 0,
    1 : -1,
    2 : -2,
}
MTRANSxvalores = {
    "Bike": 2,
    "Walking": 1,
    "Public_Transportation": 0,
    "Motorbike": -1,
    "Automobile": -2,
}
def encontrar_nivel(puntos_nivel):
    if isinstance(puntos_nivel, float):
      puntos_nivel = int(puntos_nivel)
    if -11 <= puntos_nivel <= -7:
        return "Beginner"
    elif -6 <= puntos_nivel <= -2:
        return "Intermediate"
    elif -1 <= puntos_nivel <= 3:
        return "Expert"
    else:
        return "Desconocido"

In [155]:
train.drop(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'CALC'], axis=1, inplace=True)
test.drop(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight', 'FAVC', 'FCVC', 'NCP', 'CAEC', 'SMOKE', 'CH2O', 'SCC', 'CALC'], axis=1, inplace=True)

In [156]:
train['FAF'] = train['FAF'].apply(lambda x: TUExvalores.get(int(x), 0))
train['TUE'] = train['TUE'].apply(lambda x: FAFxvalores.get(int(x), 2))
train['NObeyesdad'] = train['NObeyesdad'].replace(estadoxvalores)
train['MTRANS'] = train['MTRANS'].replace(MTRANSxvalores)
train['Puntos_Nivel'] = train['MTRANS'] + train['NObeyesdad'] + train['TUE'] + train['FAF']

In [157]:
test['FAF'] = test['FAF'].apply(lambda x: TUExvalores.get(int(x), 0))
test['TUE'] = test['TUE'].apply(lambda x: FAFxvalores.get(int(x), 2))
test['NObeyesdad'] = test['NObeyesdad'].replace(estadoxvalores)
test['MTRANS'] = test['MTRANS'].replace(MTRANSxvalores)

In [203]:
from sklearn.linear_model import LinearRegression
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error, r2_score

X_train = train.drop('Puntos_Nivel', axis=1) # datos previos
Y_train = train['Puntos_Nivel'] # resultado

# Crear una instancia del modelo de regresión lineal
model = LinearRegression()
# Entrenar el modelo utilizando los datos de entrenamiento codificados
model.fit(X_train, Y_train)
# Realizar predicciones en el conjunto de datos de prueba codificado
Y_pred = model.predict(test)

In [204]:
Y_pred = Y_pred.tolist()
Y_pred = pd.DataFrame(Y_pred, columns=['Nivel'])
# Y_pred= Y_pred['Nivel'].map(encontrar_nivel)

In [205]:
test2 = pd.read_csv('test.csv') # test data with expected results
test2['Puntos_Nivel'] = test2['MTRANS'].map(MTRANSxvalores) + test2['NObeyesdad'].map(estadoxvalores) + test2['TUE'].apply(lambda x: TUExvalores.get(int(x), 0)) + test2['FAF'].apply(lambda x: FAFxvalores.get(int(x), 2))
# expected = test2['Puntos_Nivel'].map(encontrar_nivel)

In [211]:
# Calcular el error cuadrático medio
mse = mean_squared_error(test2['Puntos_Nivel'], Y_pred)

# Calcular el coeficiente de determinación
r2 = r2_score(test2['Puntos_Nivel'], Y_pred)

# Imprimir las métricas de evaluación
def toPercent(number):
  result = number * 100 / 1
  return format(result, ".2f") + "%"

print("Error cuadrático medio:", toPercent(mse))
print("Coeficiente de determinación:", toPercent(r2))

Error cuadrático medio: 0.69%
Coeficiente de determinación: 97.18%
